## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [9]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [11]:
#
num_classes = 10

input_shape = x_train.shape[1:]

input_layer = Input(input_shape)

conv_11 = Conv2D(32, (3,3), padding='same')(input_layer)
conv_12 = Conv2D(32, (3,3), padding='same')(conv_11)
maxpooling_13 = MaxPooling2D((2,2))(conv_12)
conv_14 = Conv2D(64, (3,3), padding='same')(maxpooling_13)
conv_15 = Conv2D(64, (3,3), padding='same')(conv_14)
maxpooling_16 = MaxPooling2D((2,2))(conv_15)

conv_21 = Conv2D(32, (3,3), padding='same')(input_layer)
#conv_22 = Conv2D(32, (3,3), padding='same')(conv_21)
maxpooling_23 = MaxPooling2D((2,2))(conv_21)
conv_24 = Conv2D(64, (3,3), padding='same')(maxpooling_23)
#conv_25 = Conv2D(64, (3,3), padding='same')(conv_24)
maxpooling_26 = MaxPooling2D((2,2))(conv_24)

add_1 = Add()([maxpooling_16, maxpooling_26])

conv_31 = Conv2D(128, (3,3))(add_1)
conv_32 = Conv2D(128, (3,3))(conv_31)
maxpooling_33 = MaxPooling2D((2,2))(conv_32)

flat = Flatten()(maxpooling_33)

bn_1 = BatchNormalization()(flat)
dense_2 = Dense(512, activation="relu")(bn_1)
drop_3 = Dropout(0.25)(dense_2)

bn_4 = BatchNormalization()(drop_3)
dense_5 = Dense(128, activation="relu")(bn_4)
drop_6 = Dropout(0.25)(dense_5)

bn_7 = BatchNormalization()(drop_6)
dense_8 = Dense(32, activation="relu")(bn_7)
drop_9 = Dropout(0.25)(dense_8)

out = Dense(num_classes, activation="softmax")(drop_9)

model = keras.models.Model(inputs=[input_layer], outputs=[out])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 32, 32, 32)   896         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 32, 32, 32)   9248        conv2d_23[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D) (None, 16, 16, 32)   0           conv2d_24[0][0]                  
__________________________________________________________________________________________________
conv2d_27 

In [12]:
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [13]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
epochs = 30 # 訓練的 epochs 數量

In [15]:
model.compile(loss='categorical_crossentropy',
          optimizer=RMSprop(),
          metrics=['accuracy'])

#model = build_model()
model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size), 
                    epochs=epochs, 
                    steps_per_epoch=(x_train.shape[0]*3 // batch_size),
                    verbose=1, 
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0811 19:53:10.345000  1784 deprecation_wrapper.py:119] From C:\Users\Hui\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0811 19:53:10.358000  1784 deprecation_wrapper.py:119] From C:\Users\Hui\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0811 19:53:10.447000  1784 deprecation.py:323] From C:\Users\Hui\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
2343/2343 [==============================] - 471s 201ms/step - loss: 1.6545 - acc: 0.4042 - val_loss: 1.2497 - val_acc: 0.5423
Epoch 2/30
2343/2343 [==============================] - 469s 200ms/step - loss: 1.3069 - acc: 0.5423 - val_loss: 1.0674 - val_acc: 0.6278
Epoch 3/30
2343/2343 [==============================] - 466s 199ms/step - loss: 1.1806 - acc: 0.5938 - val_loss: 1.0109 - val_acc: 0.6535
Epoch 4/30
2343/2343 [==============================] - 465s 199ms/step - loss: 1.1035 - acc: 0.6227 - val_loss: 0.9638 - val_acc: 0.6846
Epoch 5/30
2343/2343 [==============================] - 469s 200ms/step - loss: 1.0556 - acc: 0.6429 - val_loss: 0.9501 - val_acc: 0.6823
Epoch 6/30
2343/2343 [==============================] - 465s 199ms/step - loss: 1.0155 - acc: 0.6565 - val_loss: 0.8288 - val_acc: 0.7148
Epoch 7/30
2343/2343 [==============================] - 472s 202ms/step - loss: 0.9782 - acc: 0.6704 - val_loss: 0.8646 - val_acc: 0.7056
Epoch 8/30
2343/2343 [============